In [2]:
import pandas as pd

df = pd.read_json('contacts.json')

In [3]:
df.head()

,Id,Email,Phone,Contacts,OrderId
0,0,gkzAbIy@qq.com,,1,
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov
2,2,,9125983679,0,
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa
4,4,,300364407,2,


In [4]:
%%time

df = df.join(df[df['Email'] != ''].groupby('Email')['Id'].apply(list), on='Email', rsuffix='EmailRelated')
df = df.join(df[df['Phone'] != ''].groupby('Phone')['Id'].apply(list), on='Phone', rsuffix='PhoneRelated')
df = df.join(df[df['OrderId'] != ''].groupby('OrderId')['Id'].apply(list), on='OrderId', rsuffix='OrderIdRelated')
df

Wall time: 55.7 s


,Id,Email,Phone,Contacts,OrderId,IdEmailRelated,IdPhoneRelated,IdOrderIdRelated
0,0,gkzAbIy@qq.com,,1,,[0],NaN,NaN
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,NaN,[1],"[1, 2458, 476346]"
2,2,,9125983679,0,,NaN,"[2, 348955]",NaN
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,[3],NaN,[3]
4,4,,300364407,2,,NaN,[4],NaN
...,...,...,...,...,...,...,...,...
499995,499995,,10072395382,2,whJlUOGNqjxCRzmIRdURQvlNv,NaN,[499995],[499995]
499996,499996,,571709661031,4,JqIXOONvrwvJxZqNxCYHqnrKS,NaN,[499996],[499996]
499997,499997,,4541459979,2,beXCZSzcHaBwAYoDcpQqjuAFO,NaN,[499997],[499997]
499998,499998,RzSDsyH@hotmail.com,98947185431,1,ehjeFACGiwrERQxbziMxwOWku,[499998],[499998],"[121111, 499998]"


In [ ]:
df['IdEmailRelated'] = df['IdEmailRelated'].apply(lambda d: d if isinstance(d, list) else [])
df['IdPhoneRelated'] = df['IdPhoneRelated'].apply(lambda d: d if isinstance(d, list) else [])
df['IdOrderIdRelated'] = df['IdOrderIdRelated'].apply(lambda d: d if isinstance(d, list) else [])
df['Related'] = (df['IdEmailRelated'] + df['IdPhoneRelated'] + df['IdOrderIdRelated']).apply(set).apply(list)
df

In [ ]:
df = df.set_index('Id')
df

In [3]:
df.head()

,Email,Phone,Contacts,OrderId,IdEmailRelated,IdPhoneRelated,IdOrderIdRelated,Related
Id,,,,,,,,
0,gkzAbIy@qq.com,,1,,[0],[],[],[0]
1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,[],[1],"[1, 2458, 476346]","[476346, 1, 2458]"
2,,9125983679,0,,[],"[2, 348955]",[],"[2, 348955]"
3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,[3],[],[3],[3]
4,,300364407,2,,[],[4],[],[4]


In [4]:
id_ids_dict = df.to_dict()['Related']
id_contact_dict = df.to_dict()['Contacts']

In [5]:
id_ids_tmp_dict = {}

while True:
    for key, value in id_ids_dict.items():
        for v in value:
            id_ids_dict[key] = sorted(list(set(id_ids_dict[key] + id_ids_dict[v])))
    print(id_ids_tmp_dict == id_ids_dict)
    if id_ids_tmp_dict == id_ids_dict:
        break
    else:
        id_ids_tmp_dict = id_ids_dict.copy()

False
False
False
False
True


In [6]:
id_answer_dict = {}

for key, value in id_ids_dict.items():
    contacts = 0
    for k in value:
        contacts = contacts + id_contact_dict[k]
    id_answer_dict[key] = '-'.join(list(map(str, value))) + ', ' + str(contacts)

In [7]:
df['ticket_trace/contact'] = df.index.map(id_answer_dict)
df = df.reset_index()
df = df.rename(columns={'Id': 'ticket_id'})
df.head()

,ticket_id,Email,Phone,Contacts,OrderId,IdEmailRelated,IdPhoneRelated,IdOrderIdRelated,Related,ticket_trace/contact
0,0,gkzAbIy@qq.com,,1,,[0],[],[],[0],"0, 1"
1,1,,329442681752,4,vDDJJcxfLtSfkooPhbYnJdxov,[],[1],"[1, 2458, 476346]","[476346, 1, 2458]","1-2458-98519-115061-140081-165605-476346, 12"
2,2,,9125983679,0,,[],"[2, 348955]",[],"[2, 348955]","2-159312-322639-348955, 4"
3,3,mdllpYmE@gmail.com,,0,bHquEnCbbsGLqllwryxPsNOxa,[3],[],[3],[3],"3, 0"
4,4,,300364407,2,,[],[4],[],[4],"4, 2"


In [8]:
header = ['ticket_id', 'ticket_trace/contact']
df.to_csv('output.csv', columns=header, index=False)